In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="ticks", color_codes=True)
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import missingno as msno
from xgboost import XGBRegressor
import pkg_resources
from autoviz.AutoViz_Class import AutoViz_Class
from joblib import dump

C:\Users\mmm\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\compat.py:105: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


In [13]:
#Carregamento dos dados

df = pd.read_csv("destinos_turisticos_angola.csv")
dl = pd.read_csv("dados_teste_tella.csv")

In [3]:
display(df)

,categoria_destino,sentimento_avaliacao,tipo_viajante,classificacao_media,num_avaliacoes,preco_transporte_medio,preco_hospedagem_medio,preco_alimentacao_medio,preco_lazer_medio,indice_sazonalidade,pontuacao_popularidade,fator_sustentabilidade,latitude,longitude
0,praia,positivo,casal,4.5,300,20000,150000,20000,15000,1.3,85,4.5,-8.9,13.2
1,cultural,neutro,solo,4.0,120,15000,80000,15000,9000,1.1,70,4.0,-8.8,13.3
2,eco-turismo,negativo,familia,3.8,400,18000,90000,14000,11000,1.0,60,4.7,-9.0,14.0
3,cidade,positivo,negocios,4.2,500,25000,120000,21000,18000,1.4,90,4.8,-8.7,13.9
4,montanha,neutro,aventura,4.1,220,30000,140000,17000,22000,1.2,75,4.3,-14.5,14.9


In [6]:
# Limpar espaços nos nomes das colunas
df.columns = df.columns.str.strip()

# Remover colunas desnecessárias
df = df.drop(columns=['id_destino', 'nome_destino', 'data_coleta'], axis=1, errors='ignore')


In [4]:
# Faz o print das primeiras dez linhas

print(df.head(10))

  categoria_destino  classificacao_media  num_avaliacoes sentimento_avaliacao  \
0           praia           4.82                749              neutro         
1     eco-turismo           4.14               1892            negativo         
2          cidade           4.15               3310            positivo         
3        cultural           4.95               4165            negativo         
4          cidade           3.07                422              neutro         
5          cidade           4.31               2980            negativo         
6        montanha           3.45               3044            positivo         
7        cultural           3.33               1893            negativo         
8     eco-turismo           4.19               2740            negativo         
9     eco-turismo           3.91                376            positivo         

  tipo_viajante  preco_transporte_medio  preco_hospedagem_medio  \
0    aventura            29860.78        

In [14]:
# O atributo df.info retorna uma as informacoes da nossa tabela.

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_destino               50 non-null     int64  
 1   nome_destino             50 non-null     object 
 2   categoria_destino        50 non-null     object 
 3   classificacao_media      50 non-null     float64
 4   num_avaliacoes           50 non-null     int64  
 5   sentimento_avaliacao     50 non-null     object 
 6   tipo_viajante            50 non-null     object 
 7   preco_transporte_medio   50 non-null     float64
 8   preco_hospedagem_medio   50 non-null     float64
 9   preco_alimentacao_medio  50 non-null     float64
 10  preco_lazer_medio        50 non-null     float64
 11  indice_sazonalidade      50 non-null     float64
 12  pontuacao_popularidade   50 non-null     float64
 13  fator_sustentabilidade   50 non-null     float64
 14  custo_total_previsto     50 

In [7]:
#Carregamento do arquivo csv no autoviz
AV = AutoViz_Class()
filename = "destinos_turisticos_angola.csv"
df = AV.AutoViz(filename, sep=',', depVar='', verbose=0, chart_format='svg')


Shape of your Data Set loaded: (50, 18)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    18 Predictors classified...
        2 variable(s) removed since they were ID or low-information variables
    All variables classified into correct types.


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
id_destino,int64,0.000000,100,1.000000,50.000000,Possible ID column: drop before modeling step.
nome_destino,object,0.000000,100,,,Possible ID column: drop before modeling step.
categoria_destino,object,0.000000,14,,,No issue
classificacao_media,float64,0.000000,NA,3.010000,4.980000,No issue
num_avaliacoes,int64,0.000000,98,376.000000,4650.000000,No issue
sentimento_avaliacao,object,0.000000,6,,,No issue
tipo_viajante,object,0.000000,10,,,No issue
preco_transporte_medio,float64,0.000000,NA,17320.560000,140204.890000,No issue
preco_hospedagem_medio,float64,0.000000,NA,21518.480000,244856.700000,No issue
preco_alimentacao_medio,float64,0.000000,NA,10445.840000,94611.980000,No issue


Number of All Scatter Plots = 66
All Plots done
Time to run AutoViz = 9 seconds 

 ###################### AUTO VISUALIZATION Completed ########################


In [15]:
# Modelo Escolhido O modelo XGBoost
df = df.drop(columns=['id_destino', 'nome_destino', 'data_coleta'])
cat_features = ['categoria_destino', 'sentimento_avaliacao', 'tipo_viajante']
num_features = [col for col in df.columns if col not in cat_features + ['custo_total_previsto']]
X = df.drop(columns=['custo_total_previsto'])
y = df['custo_total_previsto']


# Pipeline de pré-processamento
preprocessor = ColumnTransformer([
    ('num', MinMaxScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(objective='reg:squarederror', random_state=42))
])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [3, 6, 10],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__subsample': [0.7, 1]
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f'MSE: {mse:.2f}')
print(f'R²: {r2:.2f}')
print(f'Melhores parâmetros: {grid_search.best_params_}')


MSE: 1683595952.40
R²: 0.74
Melhores parâmetros: {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 200, 'model__subsample': 0.7}


In [17]:

dump(best_model, 'modelo_tella.joblib')  # salvar modelo completo (pipeline + XGBoost)


['modelo_tella.joblib']